In [ ]:
!pip install dask
!pip install modin
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 MB 10.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
from time import time

ray.init(ignore_reinit_error=True)

def read_file_with_time(lib, file_path):
    start_time = time()
    if lib == 'pandas':
        df = pd.read_csv(file_path)
    elif lib == 'dask':
        df = dd.read_csv(file_path)
        df = df.compute()
    elif lib == 'modin':
        df = mpd.read_csv(file_path)
    elif lib == 'ray':
        df = ray.data.read_csv(file_path)
        df = df.to_pandas()
    end_time = time()
    return df, end_time - start_time

# file paths
file_paths = [
    '/content/drive/My Drive/pasantia/hmnist_28_28_RGB.csv',
    '/content/drive/My Drive/pasantia/hmnist_28_28_L.csv',
    '/content/drive/My Drive/pasantia/hmnist_8_8_RGB.csv',
    '/content/drive/My Drive/pasantia/hmnist_8_8_L.csv',
    '/content/drive/My Drive/pasantia/HAM10000_metadata.csv'
]

for file_path in file_paths:
    print(f"\nProcesando archivo: {file_path}")
    for lib in ['pandas', 'dask', 'modin', 'ray']:
        try:
            df, time_taken = read_file_with_time(lib, file_path)
            print(f"Tiempo de lectura usando {lib}: {time_taken:.2f} segundos")
        except Exception as e:
            print(f"Error al leer con {lib}: {e}")


2024-11-11 19:55:12,331	INFO worker.py:1649 -- Calling ray.init() again after it has already been called.



Procesando archivo: /content/drive/My Drive/pasantia/hmnist_28_28_RGB.csv
Tiempo de lectura usando pandas: 3.10 segundos
Tiempo de lectura usando dask: 5.32 segundos
Tiempo de lectura usando modin: 6.69 segundos


2024-11-11 19:55:28,684	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-11-11_19-42-40_307505_987/logs/ray-data
2024-11-11 19:55:28,686	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(87) 1: 0.00 row [00:00, ? row/s]

...

2024-11-11 19:57:06,552	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-11-11_19-42-40_307505_987/logs/ray-data
2024-11-11 19:57:06,554	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]


Tiempo de lectura usando pandas: 0.99 segundos
Tiempo de lectura usando dask: 0.08 segundos
Tiempo de lectura usando modin: 0.08 segundos


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(4) 1: 0.00 row [00:00, ? row/s]

Tiempo de lectura usando ray: 0.39 segundos


In [ ]:
import yaml

for i, file_path in enumerate(file_paths):
    # column cleaning
    df.columns = df.columns.str.replace(r'\s+', '_', regex=True)
    df.columns = df.columns.str.replace(r'[^\w\s]', '', regex=True)

    schema = {
        'columns': list(df.columns),
        'separator': '|'
    }

# save to google drive
    yaml_file_path = f'/content/drive/My Drive/schema_file_{i+1}.yaml'
    with open(yaml_file_path, 'w') as yaml_file:
        yaml.dump(schema, yaml_file)

    print(f"Archivo {yaml_file_path} creado.")


Archivo /content/drive/My Drive/schema_file_1.yaml creado.
Archivo /content/drive/My Drive/schema_file_2.yaml creado.
Archivo /content/drive/My Drive/schema_file_3.yaml creado.
Archivo /content/drive/My Drive/schema_file_4.yaml creado.
Archivo /content/drive/My Drive/schema_file_5.yaml creado.


In [ ]:
for i, file_path in enumerate(file_paths):
    yaml_file_path = f'/content/drive/My Drive/schema_file_{i+1}.yaml'
    with open(yaml_file_path, 'r') as yaml_file:
        schema_loaded = yaml.load(yaml_file, Loader=yaml.FullLoader)
# Validate
    print(f"\nValidando esquema para archivo {file_path}")
    if len(df.columns) == len(schema_loaded['columns']) and list(df.columns) == schema_loaded['columns']:
        print("Validación exitosa: Los nombres de las columnas coinciden con el esquema.")
    else:
        print("Error de validación: Los nombres de las columnas no coinciden con el esquema.")



Validando esquema para archivo /content/drive/My Drive/pasantia/hmnist_28_28_RGB.csv
Validación exitosa: Los nombres de las columnas coinciden con el esquema.

Validando esquema para archivo /content/drive/My Drive/pasantia/hmnist_28_28_L.csv
Validación exitosa: Los nombres de las columnas coinciden con el esquema.

Validando esquema para archivo /content/drive/My Drive/pasantia/hmnist_8_8_RGB.csv
Validación exitosa: Los nombres de las columnas coinciden con el esquema.

Validando esquema para archivo /content/drive/My Drive/pasantia/hmnist_8_8_L.csv
Validación exitosa: Los nombres de las columnas coinciden con el esquema.

Validando esquema para archivo /content/drive/My Drive/pasantia/HAM10000_metadata.csv
Validación exitosa: Los nombres de las columnas coinciden con el esquema.


In [ ]:
for i, file_path in enumerate(file_paths):
# compress it
    output_file_path = f'/content/drive/My Drive/output_file_{i+1}.csv.gz'
    df.to_csv(output_file_path, sep='|', index=False, compression='gzip')
    print(f"Archivo comprimido guardado en Google Drive como: {output_file_path}")


Archivo comprimido guardado en Google Drive como: /content/drive/My Drive/output_file_1.csv.gz
Archivo comprimido guardado en Google Drive como: /content/drive/My Drive/output_file_2.csv.gz
Archivo comprimido guardado en Google Drive como: /content/drive/My Drive/output_file_3.csv.gz
Archivo comprimido guardado en Google Drive como: /content/drive/My Drive/output_file_4.csv.gz
Archivo comprimido guardado en Google Drive como: /content/drive/My Drive/output_file_5.csv.gz


In [ ]:
import os

for i, file_path in enumerate(file_paths):
    num_rows = df.shape[0]
    num_columns = df.shape[1]
    file_size = os.path.getsize(f'/content/drive/My Drive/output_file_{i+1}.csv.gz') / (1024 * 1024)  # Tamaño en MB

    print(f"\nResumen para archivo {file_path}:")
    print(f"Número total de filas: {num_rows}")
    print(f"Número total de columnas: {num_columns}")
    print(f"Tamaño del archivo (MB): {file_size:.2f}")



Resumen para archivo /content/drive/My Drive/pasantia/hmnist_28_28_RGB.csv:
Número total de filas: 10015
Número total de columnas: 7
Tamaño del archivo (MB): 0.07

Resumen para archivo /content/drive/My Drive/pasantia/hmnist_28_28_L.csv:
Número total de filas: 10015
Número total de columnas: 7
Tamaño del archivo (MB): 0.07

Resumen para archivo /content/drive/My Drive/pasantia/hmnist_8_8_RGB.csv:
Número total de filas: 10015
Número total de columnas: 7
Tamaño del archivo (MB): 0.07

Resumen para archivo /content/drive/My Drive/pasantia/hmnist_8_8_L.csv:
Número total de filas: 10015
Número total de columnas: 7
Tamaño del archivo (MB): 0.07

Resumen para archivo /content/drive/My Drive/pasantia/HAM10000_metadata.csv:
Número total de filas: 10015
Número total de columnas: 7
Tamaño del archivo (MB): 0.07
